# Кибериммунная автономность $\\$Создание конструктивно защищённого автономного наземного транспортного средства $\\$Модуль 1

## О документе

Версия 1.03

Общая информация и модуль 1 для регионального этапа соревнований по кибериммунной автономности




## О кибериммунной разработке

Больше информации о кибериммунном подходе к разработке можно найти на [этой](https://github.com/sergey-sobolev/cyberimmune-systems/wiki/%D0%9A%D0%B8%D0%B1%D0%B5%D1%80%D0%B8%D0%BC%D0%BC%D1%83%D0%BD%D0%B8%D1%82%D0%B5%D1%82) странице 

## Сокращения

* АНТС - автономное наземного транспортное средство
* СУПА - система управления парком автомобилей, она же - AFCS - autonomous fleet control system

## О задаче 

Автономные наземные транспортные средства с точки зрения архитектуры бортовых информационных систем мало отличаются от подводных, надводных, воздушных или космических. 

Ключевые задачи, которые нужно решить на борту, включают в себя

1. получение задания на перемещение
   
2. расчёт и осуществление перемещения в заданную точку с учётом ограничений в задании и текущих координат
   
3. передача груза получателю (наиболее подходящим для заказчика образом)

### Архитектура бортовых информационных систем

![Базовая упрощённая архитектура](images/ciac-basic-arch.png)

Рис. 1. Базовая упрощённая архитектура

Такая архитектура позволяет реализовать функции автономного перемещения в условиях отсутствия кибератак.

Эта же архитектура, но с указанием передаваемых между блоками данных показана на рисунке 2, на ней дополнительно пронумерованы блоки для удобства

![Базовая упрощённая архитектура с указанием данных](images/ciac-basic-dfd.png)

Рис. 2. Диаграмма потоков данных

Взаимодействие компонентов отражено на следующей диаграмме (рис.3). Она называется диаграммой последовательности, читать её нужно сверху вниз, по стрелкам. Каждая стрелка обозначает или запрос (сплошная) или ответ (пунктирная). Стрелки пронумерованы для удобства.

![Базовый сценарий](images/basic-scenario.png)

Рис. 3. Базовый сценарий

## Задания модуля 

### Предварительная подготовка

#### Настройка СУПА

Запустите систему управления парком автомобилей - она вам пригодится для наблюдения за движением вашего автономного наземного транспортного средства по ходу работы над всеми четырьмя модулями задания.
Для этого откройте блокнот cyberimmunity--autonomous-car-extras.ipynb и следуйте инструкциям.

Если всё получилось, то установите в ячейке ниже значение переменной afcs_present в True, если нет - оставьте False

In [1]:
afcs_present = True

#### Установка необходимых для работы пакетов

Прежде чем далее работать с кодом, установим важные пакеты в наше окружение (пакеты и их версии указаны в файле requirements.txt). 

**Примечание**: этот шаг не требуется при использовании виртуальной машины, предложенной организаторами соревнований

In [2]:
# раскомментируйте строчку ниже и запустите ячейку на выполнение

#%pip install -r requirements.txt

# при наличии ошибок установите пакеты из файла requirements.txt другим подходящим способом - 
# например, как системные пакеты

### Сборка основных компонентов

В рамках этого модуля необходимо будет из отдельных программных компонентов (блоков) собрать модель АНТС. Это похоже на крупноузловую сборку автомобиля, только нашей задаче будет не крутить гайки, а настроить информационные потоки между блоками для правильной работы всей системы.

#### Частичное решение этой задачи

0. Установим необходимые для кода примера пакеты (этот шаг не требуется при использовании виртуальной машины, предложенной организаторами соревнований)
1. Создадим функциональные компоненты (сущности 1, 3, 4 и 6), при этом каждый компонент будет работать в отдельном процессе
2. Реализуем передачу данных между компонентами
3. Реализуем получение маршрутного задания по незащищённому каналу и движение по маршруту

Важные особенности реализации:
- В качестве интерфейса взаимодействия используем очереди сообщений, у каждой сущности есть своя «персональная» очередь, ассоциированная с ней
- Внешние по отношению компоненты будут использовать этот же способ передачи данных
- Чтобы компоненты могли отправить свои сообщения в нужную очередь, создадим аналог "адресной книги" - по имени компонента в этой "книге" будет выдаваться очередь для сообщений 

1. Создадим функциональные компоненты (блоки 1, 3, 4 и 6), при этом каждый блок будет работать в отдельном процессе
2. Реализуем передачу данных между блоками
3. Реализуем получение маршрутного задания и движение по маршруту

Важные особенности реализации:

- В качестве интерфейса взаимодействия используем очереди сообщений, у каждого блока есть своя «персональная» очередь, ассоциированная с ней
  
- Внешние по отношению компоненты будут использовать этот же способ передачи данных
  
- Чтобы компоненты могли отправить свои сообщения в нужную очередь, создадим аналог "адресной книги" - по имени компонента в этой "книге" будет выдаваться очередь для сообщений.

В коде назовём компоненты следующим образом

|№ <br>на диаграмме | Название в диаграмме | Название в коде |
|--|--|--|
|1 | Связь | CommunicationGateway |
|3 | Система управления | ControlSystem |
|4 | Навигация | NavigationSystem |
|5 | Приводы груза | CargoBay |
|6 | Приводы движения | Servos |


"QueuesDirectory" - "адресная книга" - этот компонент создадим первым, чтобы все остальные могли в нём зарегистрироваться

In [3]:
from src.queues_dir import QueuesDirectory

# каталог очередей для передачи сообщений между блоками
queues_dir = QueuesDirectory()  

[ИНФО][QUEUES] создан каталог очередей


Так как у нас виртуальная машинка, то её перемещение будет рассчитываться в отдельном модуле, который находится в файле src/sitl.py (SITL - software in the loop - симулятор изменения физического состояния - в нашем случае - перемещения в пространстве), с этим модулем взаимодействуют только приводы и навигационная систем (см. рис. 4)

![Архитектура системы с симулятором](images/ciac-basic-dfd-w-sitl.png)

Рис. 4. Архитектура системы с симулятором

Теперь создадим симулятор перемещения. Как видно из блока ниже, код реализации находится в папке src, в файле sitl.py

In [4]:
from src.sitl import SITL
from geopy import Point as GeoPoint

# координата текущего положения машинки
home = GeoPoint(latitude=59.939032, longitude=30.315827) 

# идентификатор машинки (аналог VIN)
car_id = "m1" 

sitl = SITL(queues_dir=queues_dir, position=home, car_id=car_id)

[ИНФО][QUEUES] регистрируем очередь sitl
[ИНФО][SITL] симулятор создан, ID m1


Как можно заметить, мы также задали начальное местоположение машинки и идентификатор.

Далее создадим остальные функциональные блоки

**Примечание**: обратите внимание, в коде их конструкторов регистрируются собственные очереди для получения сообщений. 

Так, в конструкторе блока коммуникации можно увидеть следующий код:

```python
def __init__(self, queues_dir: QueuesDirectory):
        # вызываем конструктор базового класса
        super().__init__()

        # запоминаем каталог очередей -
        # позже он понадобится для отправки маршрутного 
        # задания в систему управления
        self._queues_dir = queues_dir

        # создаём очередь для сообщений на обработку
        self._events_q = Queue()
        self._events_q_name = self.event_source_name

        # регистрируем очередь в каталоге
>>>        self._queues_dir.register(
            queue=self._events_q, name=self._events_q_name)
```

#### Механизм решения задачи модуля 1


Для всех блоков уже создан *почти* весь необходимый для работы код. Его не нужно разрабатывать самостоятельно. Используется объектно-ориентированный подход к написанию кода. Для каждого блока, который нужно доработать, создан базовый класс, который нужно унаследовать и реализовать несколько ключевых для задачи методов (как правило, связанных с передачей данных в другой блок).

**Примечание**: реализация базовых классов находится в py файлах в папке src, рекомендуется эти файлы посмотреть, чтобы лучше понимать реализацию и логику работы блоков, допустимо использовать любой код, который там есть, однако запрещается менять любой код в папке src, если иное явно не оговорено в задании.

Ниже будут приведены примеры.

##### Настройка передачи данных в блоке "связь" (он же - "коммуникационный шлюз")

В файле src/communication_gateway.py реализован класс BaseCommunicationGateway, у которого отсутствует реализация метода _send_mission_to_consumers. Чтобы наша подсистема связи могла передавать события в систему управления, необходимо реализовать этот метод.

Это можно сделать следующим образом:

```python

from multiprocessing import Queue
from src.communication_gateway import BaseCommunicationGateway
from src.config import CONTROL_SYSTEM_QUEUE_NAME
from src.event_types import Event

class CommunicationGateway(BaseCommunicationGateway):
    """CommunicationGateway класс для реализации логики взаимодействия
    с системой планирования заданий

    Работает в отдельном процессе, поэтому создаётся как наследник класса Process
    """
    def _send_mission_to_consumers(self):
        """ метод для отправки сообщения с маршрутным заданием в систему управления """
        
        # имена очередей блоков находятся в файле src/config.py
        # события нужно отправлять в соответствие с диаграммой информационных потоков
        control_q_name = CONTROL_SYSTEM_QUEUE_NAME

        # события передаются в виде экземпляров класса Event, 
        # описание класса находится в файле src/event_types.py
        event = Event(source=BaseCommunicationGateway.event_source_name,
                      destination=control_q_name,
                      operation="set_mission", parameters=self._mission
                      )

        # поиск в каталоге нужной очереди (в данном случае - системы управления)
        control_q: Queue = self._queues_dir.get_queue(control_q_name)
        # отправка события в найденную очередь
        control_q.put(event)
                
```

Обратите внимание на определение класса Event в фале src/event_types.py - этот класс содержит описание сообщений, которые передаются между блоками:

```python
@dataclass
class Event:
    """ формат событий для обработки """
    source: str       # отправитель
    destination: str  # получатель - название очереди блока-получателя, \
                      # в которую нужно отправить сообщение
    operation: str    # чего хочет (запрашиваемое действие)
    parameters: Any   # с какими параметрами
    extra_parameters: Any = None  # дополнительные параметры

```

#### Задание: создать блок "Связь" по описанию

вставьте код из описания выше в кодовый блок и создайте экземпляр класс CommunicationGateway

In [5]:
# вставьте свой код в эту ячейку

Создадим экземпляр получившегося класса

In [7]:
communication_gateway = CommunicationGateway(queues_dir=queues_dir)

[ИНФО][QUEUES] регистрируем очередь communication
[ИНФО][COMMUNICATION] создан компонент связи


#### Задание: создать блоки "Система управления", "Навигация" по аналогии

По аналогии реализуйте передачу информации между другими блоками (системы управления, навигации и приводами) согласно диаграмме на рис. 4

In [8]:
# найдите подходящее имя очереди в файле src/config.py и добавьте в строчку ниже
#from src.config import 
from src.control_system import BaseControlSystem


class ControlSystem(BaseControlSystem):
    """ControlSystem блок расчёта управления """

    def _send_speed_and_direction_to_consumers(self, speed, direction):
        servos_q_name = None # замените на правильное название очереди
        servos_q: Queue = self._queues_dir.get_queue(servos_q_name)

        # отправка сообщения с желаемой скоростью
        event_speed = None # замените на код создания сообщения со скоростью для приводов
                           # подсказка, требуемая операция - set_speed

        # отправка сообщения с желаемым направлением
        event_direction = None # замените на код создания сообщения с направлением для приводов
                               # подсказка, требуемая операция - set_direction

        servos_q.put(event_speed)
        servos_q.put(event_direction)       


In [9]:
# найдите подходящее имя очереди в файле src/config.py и добавьте в строчку ниже
#from src.config import 
from src.navigation_system import BaseNavigationSystem


class NavigationSystem(BaseNavigationSystem):
    """ класс навигационного блока """
    def _send_position_to_consumers(self):        
        control_q_name = None # замените на правильное название очереди
        event = None # замените на код создания сообщения с координатами для системы управления 
                     # подсказка, требуемая операция - position_update
        control_q: Queue = self._queues_dir.get_queue(control_q_name)
        control_q.put(event)

In [12]:
control_system = ControlSystem(queues_dir=queues_dir)
navigation_system = NavigationSystem(queues_dir=queues_dir)

[ИНФО][QUEUES] регистрируем очередь control
[ИНФО][CONTROL] создана система управления
[ИНФО][QUEUES] регистрируем очередь navigation
[ИНФО][NAVIGATION] создан компонент навигации


Создадим экземпляр класса "Приводы"

In [13]:
from src.servos import Servos

servos = Servos(queues_dir=queues_dir)

[ИНФО][QUEUES] регистрируем очередь servos
[ИНФО][SERVOS] создан компонент сервоприводов


Теперь создадим простейшую систему планирования заданий для перевозки, в ней будем задавать координаты пункта назначения и ограничения скорости перемещения.

In [14]:
from src.mission_planner import MissionPlanner

mission_planner = MissionPlanner(queues_dir=queues_dir)

[ИНФО][QUEUES] регистрируем очередь planner
[ИНФО][MISSION PLANNER] создана система планирования заданий


Создадим новую задачу на перевозку со следующими параметрами:
- home - координаты начальной точки перемещения
- waypoints - координаты путевых точек (через них должна проехать наша машинка)
- speed_limits - скоростные ограничения для заданного отрезка пути, 0 - отрезок от 0 до 1 точки, 1 - от 1 до 2 путевой точки; если путевых точек больше, то в отсутствие скоростного ограничения для какого-то сегмента должно использоваться последнее заданное ограничение
- armed - разрешение на выезд

In [15]:
from src.mission_type import Mission, GeoSpecificSpeedLimit

mission = Mission(home=home,
                waypoints=[GeoPoint(latitude=59.9386, longitude=30.3121),
                           GeoPoint(latitude=59.9386, longitude=30.3149),
                           GeoPoint(latitude=59.9421, longitude=30.3067)
                           ],
                speed_limits=[
                    GeoSpecificSpeedLimit(0, 30),
                    GeoSpecificSpeedLimit(1, 60)
                ],
                armed=True)

#### Задание: замените координаты и скоростные ограничения в описании маршрутного задания согласно предложенным начальным и конечным точкам

In [16]:
mission_planner.set_new_mission(mission=mission)

Теперь зададим идентификатор машинки (аналог VIN номера) - он будет виден в СУПА, если эта система у вас работает.

In [17]:
car_id = "m1"

Следующим шагом запустим все компоненты и позволим системе поработать какое-то время. 

По итогам работы этого кода под блоком должен появиться текст с событиями о запуске всех подсистем, начале и окончании движения по маршруту.

In [18]:
from time import sleep
from src.config import LOG_INFO

# создаём основные блоки
sitl = SITL(queues_dir=queues_dir, position=home, car_id=car_id, log_level=LOG_INFO)        # <- симулятор движения
communication_gateway = CommunicationGateway(queues_dir=queues_dir, log_level=LOG_INFO)     # <- Связь
control_system = ControlSystem(queues_dir=queues_dir, log_level=LOG_INFO)                   # <- Система управления
navigation_system = NavigationSystem(queues_dir=queues_dir, log_level=LOG_INFO)             # <- Навигация
servos = Servos(queues_dir=queues_dir, log_level=LOG_INFO)                                  # <- Приводы


# запускаем созданные выше блоки системы, после этого они начнут обработку входящих событий
sitl.start()
navigation_system.start()
servos.start()
communication_gateway.start()
control_system.start()
mission_planner.start()


# пусть машинка немного поездит, 
# параметр sleep - время в секундах
sleep(5)

# останавливаем все компоненты
control_system.stop()
communication_gateway.stop()
mission_planner.stop()
sitl.stop()
servos.stop()
navigation_system.stop()

# дождёмся завершения работы всех компонентов
control_system.join()
communication_gateway.join()
mission_planner.join()
sitl.join()
servos.join()
navigation_system.join()

# подчистим все ресурсы для возможности повторного запуска в следующих модулях
del control_system, communication_gateway, mission_planner, sitl, servos, navigation_system

[ИНФО][QUEUES] регистрируем очередь sitl
[ИНФО][SITL] симулятор создан, ID m1
[ИНФО][QUEUES] регистрируем очередь communication
[ИНФО][COMMUNICATION] создан компонент связи
[ИНФО][QUEUES] регистрируем очередь control
[ИНФО][CONTROL] создана система управления
[ИНФО][QUEUES] регистрируем очередь navigation
[ИНФО][NAVIGATION] создан компонент навигации
[ИНФО][QUEUES] регистрируем очередь servos
[ИНФО][SERVOS] создан компонент сервоприводов
[ИНФО][SITL] [SITL] старт симуляции
[ИНФО][NAVIGATION] старт навигации
[ИНФО][SERVOS] старт блока приводов
[ИНФО][COMMUNICATION] старт системы планирования заданий
[ИНФО][CONTROL] старт системы управления
[ИНФО][MISSION PLANNER] старт системы планирования заданий
[ИНФО][MISSION PLANNER] запрошена новая задача, отправляем получателям
[ИНФО][MISSION PLANNER] новая задача отправлена в коммуникационный шлюз
[ИНФО][COMMUNICATION] получен новый маршрут, отправляем в получателям
[ИНФО][CONTROL] установлена новая задача, начинаем следовать по маршруту, текущее

Если всё сделано правильно, то на этом этапе наша виртуальная машинка уже может автономно следовать к заданным точкам. Однако не хватает важного шага - доставки груза в конечной точке маршрута.

#### Задание: дописать в системе управления метод отправки события с командой на отгрузку в конечной точке маршрута.

Сама логика определения конечной точки маршрута уже есть в базовом классе системы управления, не хватает только шага отправки события. 
Для выполнения этой задачи нужно реализовать абстрактные методы _release_cargo (разблокировать грузовой отсек, т.е. оставить груз) и _lock_cargo (заблокировать грузовой отсек, эту команду система управления отправляет в начале следования по маршруту - будет неправильно потерять груз по дороге)

В следующей ячейке измените соответствующие методы ControlSystem, чтобы система управления могла отправлять команды для блокировку и открытие грузового отсека. 

In [19]:
from src.config import CARGO_BAY_QUEUE_NAME,\
    SERVOS_QUEUE_NAME
    
class ControlSystem(BaseControlSystem):
    """ControlSystem блок расчёта управления """

    def _send_speed_and_direction_to_consumers(self, speed, direction):
        servos_q_name = SERVOS_QUEUE_NAME
        servos_q: Queue = self._queues_dir.get_queue(servos_q_name)

        # инициализация сообщения с желаемой скоростью
        # подсказка: блок Приводы ожидает команду "set_speed" с параметром в виде скорости
        event_speed = None # <-- измените эту строку!

        # отправка сообщения с желаемым направлением
        # подсказка: блок Приводы ожидает команду "set_direction" с параметром в виде направления
        event_direction = None # <-- измените эту строку!

        servos_q.put(event_speed)
        servos_q.put(event_direction)

    def _lock_cargo(self):
        """ заблокировать грузовой отсек """
        cargo_q = self._queues_dir.get_queue(CARGO_BAY_QUEUE_NAME)
        # инициализация сообщения с командой на блокировку грузового отсека
        # подсказка: блок CargoBay ожидает команду "lock_cargo" без параметров
        event = None # <-- измените эту строку!
        cargo_q.put(event)

    def _release_cargo(self):
        """ открыть грузовой отсек """
        cargo_q = self._queues_dir.get_queue(CARGO_BAY_QUEUE_NAME)
        # инициализация сообщения с командой на блокировку грузового отсека
        # подсказка: блок CargoBay ожидает команду "release_cargo" без параметров
        event = None # <-- измените эту строку!

        cargo_q.put(event)


In [20]:
# пример решения удалить перед выдачей конкурсантам!

from src.config import CARGO_BAY_QUEUE_NAME,\
    SERVOS_QUEUE_NAME
    
class ControlSystem(BaseControlSystem):
    """ControlSystem блок расчёта управления """

    def _send_speed_and_direction_to_consumers(self, speed, direction):        
        servos_q_name = SERVOS_QUEUE_NAME
        servos_q: Queue = self._queues_dir.get_queue(servos_q_name)

        # отправка сообщения с желаемой скоростью
        event_speed = Event(source=self.event_source_name,
                            destination=servos_q_name,
                            operation="set_speed",
                            parameters=speed
                            )

        # отправка сообщения с желаемым направлением
        event_direction = Event(source=self.event_source_name,
                                destination=servos_q_name,
                                operation="set_direction",
                                parameters=direction
                                )

        servos_q.put(event_speed)
        servos_q.put(event_direction)

    def _lock_cargo(self):
        """ заблокировать грузовой отсек """
        cargo_q = self._queues_dir.get_queue(CARGO_BAY_QUEUE_NAME)
        event = Event(source=CONTROL_SYSTEM_QUEUE_NAME,
                      destination=CARGO_BAY_QUEUE_NAME,
                      operation="lock_cargo",
                      parameters=None
                      )
        cargo_q.put(event)

    def _release_cargo(self):
        """ открыть грузовой отсек """
        cargo_q = self._queues_dir.get_queue(CARGO_BAY_QUEUE_NAME)
        event = Event(source=CONTROL_SYSTEM_QUEUE_NAME,
                      destination=CARGO_BAY_QUEUE_NAME,
                      operation="release_cargo",
                      parameters=None
                      )
        cargo_q.put(event)

Проверим результат. Если всё сделано правильно, то после завершения маршрута мы должны увидеть в тексте "груз оставлен"

In [21]:
from time import sleep
from src.mission_planner import MissionPlanner
from src.cargo_bay import CargoBay
from src.config import LOG_ERROR, LOG_INFO
from src.mission_type import Mission, GeoSpecificSpeedLimit
from src.system_wrapper import SystemComponentsContainer

# вспомогательные блоки для отправки данных в СУПА
from src.mission_planner_mqtt import MissionSender
from src.sitl_mqtt import TelemetrySender


mission_sender = MissionSender(
    queues_dir=queues_dir, client_id=car_id, log_level=LOG_INFO)
telemetry_sender = TelemetrySender(
    queues_dir=queues_dir, client_id=car_id, log_level=LOG_INFO)

home = GeoPoint(latitude=59.9386, longitude=30.3121)

# сократим маршрут для ускорения процесса
mission = Mission(
    home=home,
    waypoints=[home, GeoPoint(latitude=59.9421, longitude=30.3067)
               ],
    speed_limits=[
        GeoSpecificSpeedLimit(0, 60)
    ],
    armed=True)

mission_planner = MissionPlanner(
    queues_dir=queues_dir, afcs_present=afcs_present)
mission_planner.set_new_mission(mission=mission)

sitl = SITL(queues_dir=queues_dir, position=home,
            car_id=car_id, log_level=LOG_INFO, post_telemetry=afcs_present)
communication_gateway = CommunicationGateway(
    queues_dir=queues_dir, log_level=LOG_INFO)
control_system = ControlSystem(queues_dir=queues_dir, log_level=LOG_INFO)
navigation_system = NavigationSystem(
    queues_dir=queues_dir, log_level=LOG_INFO)
servos = Servos(queues_dir=queues_dir, log_level=LOG_ERROR)
cargo_bay = CargoBay(queues_dir=queues_dir, log_level=LOG_INFO)


# у нас получилось довольно много блоков, используем класс SystemComponentsContainer
# для упрощения рутинной работы с ними
system_components = SystemComponentsContainer(
    components=[
        mission_sender,
        telemetry_sender,
        sitl,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system,
        mission_planner
    ] if afcs_present else [
        sitl,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system,
        mission_planner
    ])

system_components.start()

# пусть машинка немного поездит,
# параметр sleep - время в секундах
sleep(35)

# останавливаем все компоненты
system_components.stop()

# подчистим все ресурсы для возможности повторного запуска в следующих модулях
system_components.clean()

[ИНФО][QUEUES] регистрируем очередь planner.mqtt
[ИНФО][QUEUES] регистрируем очередь sitl.mqtt
[ИНФО][QUEUES] регистрируем очередь planner
[ИНФО][MISSION PLANNER] создана система планирования заданий
[ИНФО][QUEUES] регистрируем очередь sitl
[ИНФО][SITL] симулятор создан, ID m1
[ИНФО][QUEUES] регистрируем очередь communication
[ИНФО][COMMUNICATION] создан компонент связи
[ИНФО][QUEUES] регистрируем очередь control
[ИНФО][CONTROL] создана система управления
[ИНФО][QUEUES] регистрируем очередь navigation
[ИНФО][NAVIGATION] создан компонент навигации
[ИНФО][QUEUES] регистрируем очередь servos
[ИНФО][QUEUES] регистрируем очередь cargo
[ИНФО][CARGO] создан компонент грузового отсека, отсек заблокирован
[ИНФО][MISSION_PLANNER.MQTT] старт клиента телеметрии
[ИНФО][MISSION_PLANNER.MQTT] клиент отправки маршрута создан и запущен[ИНФО][SITL.MQTT] старт клиента телеметрии


[ИНФО][SITL.MQTT] клиент отправки телеметрии создан и запущен
[ИНФО][SITL] [SITL] старт симуляции[ИНФО][CARGO] старт блока гр

На этом модуль 1 завершён, едем дальше!

Для работы над модулем 2 задания откройте блокнот с названием cyberimmunity--autonomous-car-m2.